<a href="https://colab.research.google.com/github/rodriguezhh/IA_SIDIEN/blob/main/IA_funcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


CONSTRUYENDO UN MODELO SECUENCIAL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Manipulación del Dataset

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

IMAGE_SIZE = 256
BATCH_SIZE = 64
CHANELS = 3
EPOCHS = 20

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Universidad/Tesis/Dataset/color",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Obtención de las clases y visualización

In [ ]:
class_names = dataset.class_names
class_names

Se define los datos de entrenamiento, de validación y de testeo, mediante una Función.

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):

    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)

    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

Se guardan los datos

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
print(len(train_ds), len(val_ds), len(test_ds))

Resize and rescale

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

Data augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
     ]
)

Construyendo el modelo 

In [ ]:
inputs = keras.Input(shape = (256, 256, 3))

x = resize_and_rescale(inputs)
x = data_augmentation(inputs)
x = layers.Conv2D(32, (3,3), activation="relu")(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(16, (3,3), activation="relu")(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(8, (3,3), activation="relu")(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="rmsprop",#adam
 loss="sparse_categorical_crossentropy",
 metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    batch_size = BATCH_SIZE
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print('first image to predict')
    plt.imshow(first_image)
    print('actual label', class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print("predicted label: ", class_names[np.argmax(batch_prediction[0])])